In [26]:
import pandas as pd

def process_and_save(df_in):
    # 找到所有的NaN列
    nan_cols = df_in.columns[df_in.isna().all()]

    # 初始化一个空的DataFrame来存储当前的品种数据
    df_current = pd.DataFrame()

    # 初始化一个空的列表来存储所有的品种数据
    dfs = []

    # 遍历所有的列
    for col in df_in.columns:
        # 如果当前列是NaN列，那么我们已经找到了一个新的品种
        if col in nan_cols:
            # 如果当前的品种数据不为空，那么我们将其添加到列表中
            if not df_current.empty:
                dfs.append(df_current)
            # 然后我们初始化一个新的DataFrame来存储新的品种数据
            df_current = pd.DataFrame()

        else:
            # 如果当前列不是NaN列，那么我们将其添加到当前的品种数据中
            df_current[col] = df_in[col]

    # 如果最后一个品种数据没有被添加到列表中，那么我们需要手动添加
    if not df_current.empty:
        dfs.append(df_current)

    # 现在我们有了一个包含所有品种数据的列表，我们可以将每个品种数据保存为一个单独的文件
    for df in dfs:
        # 去除表的列索引的符号和数字
        df.columns = df.columns.str.replace('\.\d+', '', regex=True)
        unique_types = df['时间'].apply(type).unique()
        if len(unique_types) > 1 or unique_types[0] != pd._libs.tslibs.timestamps.Timestamp:
            print(f"品种 {df[df.columns[1]][0]} 的 '时间' 列存在非日期格式的数据，需要进行数据清洗。")
            continue  # 如果'时间'列存在非日期格式的数据，跳过这个品种的处理
        # 把时间列设为index，只保留日期部分
        df['时间'] = pd.to_datetime(df['时间']).dt.normalize()
        df.set_index('时间', inplace=True)

# 构造文件名

        # 获取品种名
        variety_name = df[df.columns[0]][0]
        # 构造文件名
        file_name = f'../数据/各品种主连数据/{variety_name}_{df.index.min().strftime("%Y%m%d")}_to_{df.index.max().strftime("%Y%m%d")}.xlsx'


        # 保存文件
        df.to_excel(file_name, index=True)

In [122]:


def adjust_and_save(file_path):
    df = pd.read_excel(file_path,index_col=0,parse_dates=[0])
    df['月合约代码_shift'] = df['月合约代码'].shift(1)
    df['前一日收盘价'] = df['收盘价'].shift(1)
    df['切换日'] = df['月合约代码'] != df['月合约代码_shift']
    df.iloc[0:1, df.columns.get_loc('切换日')] = False
    df.loc[df['切换日'], '复权系数'] = df.loc[df['切换日'],'前一日收盘价']/ df.loc[df['切换日'],'收盘价']
    df['复权系数']=df['复权系数'].fillna(method='ffill')

    df_first_day_of_month = df[df['切换日']].copy()
    df_first_day_of_month = df_first_day_of_month.sort_values(by='月合约代码')
    df_first_day_of_month['复权系数'] = df_first_day_of_month['复权系数'].cumprod()
    df = df.reset_index().merge(df_first_day_of_month[['月合约代码', '复权系数']], on='月合约代码', how='left').set_index('时间')
    df.index = pd.to_datetime(df.index)
    
    df['复权开盘价'] = df['开盘价'] * df['复权系数_y']
    df['复权最高价'] = df['最高价'] * df['复权系数_y']
    df['复权最低价'] = df['最低价'] * df['复权系数_y']
    df['复权收盘价'] = df['收盘价'] * df['复权系数_y']
    df.index = pd.to_datetime(df.index)
    variety_name = df['期货简称'][0]  # 从文件路径中获取品种名
    # 构造文件名
    file_name = f'../数据/复权后品种行情数据/{variety_name}_{str(df.index.min().date())}_to_{str(df.index.max().date())}后复权数据.xlsx'
    print(f'{file_name}保存成功')
    print(type(df.index[0]))

    df.to_excel(file_name)

In [123]:
import os


../数据/复权后品种行情数据/20号胶主连_2022-01-04_to_2023-09-05后复权数据.xlsx保存成功
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
../数据/复权后品种行情数据/LPG主连_2022-01-04_to_2023-09-05后复权数据.xlsx保存成功
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
../数据/复权后品种行情数据/PTA主连_2022-01-04_to_2023-09-05后复权数据.xlsx保存成功
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
../数据/复权后品种行情数据/PVC主连_2022-01-04_to_2023-09-05后复权数据.xlsx保存成功
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
../数据/复权后品种行情数据/三十年国债主连_2022-01-04_to_2023-09-05后复权数据.xlsx保存成功
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
../数据/复权后品种行情数据/上证50主连_2022-01-04_to_2023-09-05后复权数据.xlsx保存成功
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
../数据/复权后品种行情数据/不锈钢主连_2022-01-04_to_2023-09-05后复权数据.xlsx保存成功
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
../数据/复权后品种行情数据/中证1000主连_2022-01-04_to_2023-09-05后复权数据.xlsx保存成功
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
../数据/复权后品种行情数据/中证500主连_2022-01-04_to_2023-09-05后复权数据.xlsx保存成功
<class 'pandas._libs.tslibs.timest

In [120]:
adjust_and_save(r'C:\Users\Administrator.DESKTOP-BCUJV93\Desktop\量化学习\数据\各品种主连数据\乙二醇主连_20220104_to_20230905.xlsx')

<class 'pandas._libs.tslibs.timestamps.Timestamp'>
2022-01-04
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
2022-01-04
../数据/复权后品种行情数据/乙二醇主连_2022-01-04_to_2023-09-05后复权数据.xlsx保存成功
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
